In [1]:
import pandas as pd
from collections import Counter

### read tabular result

In [2]:
resultFilepath="/data/mitsuki/out/altorf/evolve/result/GCF_000022125.1_ASM2212v1.tab2"
names_lst=["qseqid","sseqid","pident","length","mismatch","gapopen","qstart","qend","sstart","send","evalue","bitscore"]
result_df=pd.read_csv(resultFilepath,delimiter='\t',names=names_lst)
result_df["category"]=[qseqid.split('|')[0] for qseqid in result_df["qseqid"]]
print(result_df.shape)
result_df.head()

(11170, 13)


,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,category
0,F2|lcl|NC_011883.1_cds_WP_012623654.1_2,WP_012612930.1,34.328,67,32,3,342,405,20,77,0.86,31.6,F2
1,F4|lcl|NC_011883.1_cds_WP_012623654.1_2,WP_015860831.1,30.108,93,58,2,25,115,15,102,0.29,32.3,F4
2,F4|lcl|NC_011883.1_cds_WP_012623654.1_2,WP_041723756.1,100.000,13,0,0,512,524,1,13,0.48,30.8,F4
3,F4|lcl|NC_011883.1_cds_WP_041723756.1_3,WP_012623654.1,92.308,13,1,0,97,109,1,13,0.57,28.9,F4
4,F6|lcl|NC_011883.1_cds_WP_041723756.1_3,WP_014260517.1,29.114,79,49,3,27,101,60,135,0.33,29.3,F6


## count hits by category

In [3]:
thres_lst=[1,0.1,0.01,1e-3,1e-4,1e-5]
dct_lst=[]
for thres in thres_lst:
    dct={}
    dct["thres"]=thres
    filtered_df=result_df[result_df["evalue"]<thres]
    
    filtered_df=filtered_df.drop_duplicates(subset=['qseqid'])
    dct.update(Counter(filtered_df["category"]))
    dct_lst.append(dct)
count_df=pd.DataFrame(dct_lst)
count_df

,DRF2,DRF3,DRF4,DRF5,DRF6,DSF2,DSF3,DSF4,DSF5,DSF6,F2,F3,F4,F5,F6,thres
0,465.0,434.0,842.0,351.0,699,421.0,426.0,830.0,385.0,529.0,506,551,987,472,610,1.00000
1,68.0,59.0,130.0,65.0,257,71.0,51.0,127.0,50.0,76.0,90,88,170,78,96,0.10000
2,9.0,10.0,10.0,2.0,135,8.0,6.0,15.0,3.0,10.0,15,22,33,14,15,0.01000
3,1.0,NaN,2.0,NaN,84,NaN,1.0,2.0,NaN,NaN,10,9,9,3,3,0.00100
4,NaN,NaN,NaN,NaN,68,NaN,NaN,NaN,NaN,NaN,10,9,6,3,2,0.00010
5,NaN,NaN,NaN,NaN,46,NaN,NaN,NaN,NaN,NaN,8,8,4,3,2,0.00001


In [5]:
count_df=count_df.fillna(0)
name_lst=[]
for prefix in ["F","DRF","DSF"]:
    for i in range(2,7):
        name=prefix+str(i)
        name_lst.append(name)
count_df[name_lst]=count_df[name_lst].astype(int)
count_df=count_df[["thres"]+name_lst]
count_df

,thres,F2,F3,F4,F5,F6,DRF2,DRF3,DRF4,DRF5,DRF6,DSF2,DSF3,DSF4,DSF5,DSF6
0,1.00000,506,551,987,472,610,465,434,842,351,699,421,426,830,385,529
1,0.10000,90,88,170,78,96,68,59,130,65,257,71,51,127,50,76
2,0.01000,15,22,33,14,15,9,10,10,2,135,8,6,15,3,10
3,0.00100,10,9,9,3,3,1,0,2,0,84,0,1,2,0,0
4,0.00010,10,9,6,3,2,0,0,0,0,68,0,0,0,0,0
5,0.00001,8,8,4,3,2,0,0,0,0,46,0,0,0,0,0
